# Initialize Env

In [1]:
from my_utils import init_env
import mediapy as media
env = init_env(wandb_enabled=False)

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /research/rs4tmr/robosuite/scripts/setup_macros.py (macros.py:55)


Initalized env with init_env
fix_object:None
active_rewards: ['r', 'g', 'l', 'h']


In [2]:
env.action_space.sample()

array([ 0.40042293,  0.555742  ,  0.89154464,  0.24164245, -0.92143774,
        0.5926629 ,  0.8203387 ], dtype=float32)

In [3]:
obs = env.reset()
print(obs[0].shape)
#env.step(env.action_space.sample())

(39,)


In [6]:
env.step(env.action_space.sample())

(array([ 1.82330929e-01, -1.60444739e-01,  8.60049060e-01,  1.01499951e-04,
         2.86367925e-06,  5.67199675e-01,  8.23580305e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -4.62736731e-01, -1.75366954e+00,
         2.46353836e+00, -2.25814027e+00, -1.60262569e+00, -1.97096943e+00,
        -4.55423569e-03,  4.66794841e-02, -8.24490000e-02,  3.19293302e-02,
        -1.23902292e-02, -4.18278282e-02, -1.62235140e-01, -1.27381766e-01,
         9.92336382e-01, -9.99325406e-01,  3.11289610e-02,  1.68813781e-02,
        -9.73345478e-03,  2.53249655e-02, -1.56067148e-02,  1.46576935e-02,
        -1.13620419e-02,  2.86288036e-03,  3.75209867e-02,  1.68853982e-01,
        -3.12082981e-01, -2.21873466e-02,  1.68822876e-01, -3.12543534e-01,
        -2.21542338e-02, -1.08678359e+00,  2.24025552e+00, -1.06142166e+00,
        -1.08701484e+00,  2.22533782e+00, -1.06056026e+00]),
 np.float64(0.0001208024093

# Get images

In [7]:
env.image_states

/root/anaconda3/envs/rs4tmr/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.image_states to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.image_states` for environment variables or `env.get_wrapper_attr('image_states')` that will search the reminding wrappers.
  logger.warn(


{'agentview_image': array([[[203, 190, 182],
         [211, 198, 189],
         [208, 196, 185],
         ...,
         [186, 177, 168],
         [191, 182, 173],
         [187, 178, 169]],
 
        [[203, 190, 182],
         [208, 195, 186],
         [210, 197, 187],
         ...,
         [191, 182, 173],
         [193, 184, 175],
         [179, 170, 161]],
 
        [[205, 193, 185],
         [205, 192, 184],
         [208, 195, 186],
         ...,
         [203, 195, 186],
         [191, 183, 174],
         [184, 175, 166]],
 
        ...,
 
        [[117, 115, 111],
         [118, 116, 112],
         [118, 116, 112],
         ...,
         [117, 115, 111],
         [117, 115, 111],
         [116, 114, 111]],
 
        [[118, 117, 113],
         [118, 116, 113],
         [118, 116, 113],
         ...,
         [118, 116, 112],
         [117, 116, 112],
         [117, 115, 112]],
 
        [[118, 116, 113],
         [118, 116, 112],
         [117, 115, 112],
         ...,
         

# Get observation according key name

In [1]:
env.reset()
obs = env.step(env.action_space.sample())
env.get_obs_according_to_key(obs[0],'robot0_joint_pos_cos')

NameError: name 'env' is not defined

# Model Checkpoint load and Rollout

In [1]:
import torch
import gymnasium as gym
import numpy as np
import mediapy as media
# 필요한 클래스 및 함수 임포트
from cleanrl.cleanrl.ppo_continuous_action import Agent, Args, jesnk_make_env
import cv2

def load_model_and_evaluate(model_path, env_id="HalfCheetah-v4", num_episodes=10, seed=1, gamma=0.99):
    """
    저장된 모델을 불러와 환경에서 평가를 수행하는 함수
    """
    visualize = True
    frames = []
    
    # Argument 설정
    args = Args()
    args.env_id = env_id
    args.seed = seed
    args.gamma = gamma

    # 환경 생성
    env = gym.vector.SyncVectorEnv(
        [jesnk_make_env(args.env_id, i, args.capture_video, "evaluation", args.gamma, wandb_enabled=False, active_rewards='r',active_image=True) for i in range(1)]
    )
    
    # 디바이스 설정 (cuda가 가능하면 cuda 사용)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if device == torch.device("cuda"):
        print("Using CUDA")
    else :
        assert device == torch.device("cpu")

    # Agent 초기화 및 모델 불러오기
    agent = Agent(env).to(device)
    agent.load_state_dict(torch.load(model_path, map_location=device))
    agent.eval()  # 평가 모드로 전환

    # 평가 수행
    total_rewards = []
    for episode in range(num_episodes):
        obs, _ = env.reset(seed=args.seed)
        obs = torch.Tensor(obs).to(device)
        done = False
        episode_reward = 0
        
        image_frame = env.envs[0].image_states['agentview_image']
        image_frame = np.array(image_frame[::-1, :, :], dtype=np.uint8)  # 명시적으로 numpy 배열로 변환
        frames.append(image_frame)
        # convert image_frame cv2 image
        #image_frame = cv2.cvtColor(image_frame, cv2.COLOR_RGB2BGR)
        
        frames.append(image_frame)
        
        
        while not done:

            with torch.no_grad():
                obs = torch.Tensor(obs).to(device)
                action, _, _, _ = agent.get_action_and_value(obs)
            obs, reward, terminations, truncations, infos = env.step(action.cpu().numpy())
            done = np.logical_or(terminations, truncations).any()
            episode_reward += reward[0]  # 첫 번째 환경의 보상 합산
            
            # 새로운 프레임 가져오기 및 변환
            image_frame = env.envs[0].image_states['agentview_image']
            image_frame = np.array(image_frame[::-1, :, :], dtype=np.uint8)  # numpy 배열로 변환

            #image_frame = cv2.cvtColor(image_frame, cv2.COLOR_RGB2BGR)
            # draw text on image_frame episode reward, reward, small text
            cv2.putText(image_frame, f"Episode Reward: {episode_reward:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(image_frame, f"Reward: {reward[0]:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            frames.append(image_frame)
            

        print(f"Episode {episode + 1}: Total Reward: {episode_reward}")
        total_rewards.append(episode_reward)

    env.close()

    # 평균 리턴 출력
    avg_reward = np.mean(total_rewards)
    print(f"Average Reward over {num_episodes} episodes: {avg_reward}")
    
    media.show_video(frames, fps=30)



[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /research/rs4tmr/robosuite/scripts/setup_macros.py (macros.py:55)


In [2]:
# 저장된 모델 불러와 평가하기

model_path = "/research/rs4tmr/cleanrl/cleanrl/runs/tr__ppo_continuous_action__s1__2024-09-25_08-28-09/ppo_continuous_action.cleanrl_model"  # 모델 경로 지정
load_model_and_evaluate(model_path, env_id="HalfCheetah-v4", num_episodes=1)

Initalized env with init_env
fix_object:False
active_rewards: ['r']
Using CUDA


/tmp/ipykernel_822925/2007038180.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent.load_state_dict(torch.load(model_path, map_location=device))
/root/anaconda3/envs

Episode 1: Total Reward: 1.962892069948548
Average Reward over 1 episodes: 1.962892069948548


This browser does not support the video tag.


In [1]:
import robosuite as suite
from robosuite.wrappers import GymWrapper

from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor


rsenv = suite.make(
    "TmrPickPlaceCan",
    robots="UR5e",  # use UR5e robot
    use_camera_obs=False,  # use pixel observations
    has_offscreen_renderer=True,  # needed if using pixel obs
    has_renderer=True,  # make sure we can render to the screen
    reward_shaping=True,  # use dense rewards
    control_freq=20,  # control should happen fast enough so that simulation looks smooth
)

# full_observable_list = [
#     'robot0_joint_pos', 
#     'robot0_joint_pos_cos', 
#     'robot0_joint_pos_sin', 
#     'robot0_joint_vel', 
#     'robot0_eef_pos', 
#     'robot0_eef_quat', 
#     'robot0_eef_vel_lin', 
#     'robot0_eef_vel_ang', 
#     'robot0_gripper_qpos', 
#     'robot0_gripper_qvel', 
#     'agentview_image', 
#     'world_pose_in_gripper', 
#     'Milk_pos', 
#     'Milk_quat', 
#     'Milk_to_robot0_eef_pos', 
#     'Milk_to_robot0_eef_quat', 
#     'Bread_pos', 
#     'Bread_quat', 
#     'Bread_to_robot0_eef_pos', 
#     'Bread_to_robot0_eef_quat', 
#     'Cereal_pos', 
#     'Cereal_quat', 
#     'Cereal_to_robot0_eef_pos', 
#     'Cereal_to_robot0_eef_quat', 
#     'Can_pos', 
#     'Can_quat', 
#     'Can_to_robot0_eef_pos', 
#     'Can_to_robot0_eef_quat',
# ]
# for observable in full_observable_list:
#     rsenv.modify_observable(observable, 'enabled', True)
#     rsenv.modify_observable(observable, 'active', True)

useless_observable_list = [
    # 'robot0_joint_pos', 
    # 'robot0_joint_pos_cos', 
    # 'robot0_joint_pos_sin', 
    # 'robot0_joint_vel', 
    # 'robot0_eef_pos', 
    # 'robot0_eef_quat', 
    # 'robot0_eef_vel_lin', 
    # 'robot0_eef_vel_ang', 
    # 'robot0_gripper_qpos', 
    # 'robot0_gripper_qvel', 
    # 'agentview_image', 
    # 'world_pose_in_gripper', 
    # 'Milk_pos', 
    # 'Milk_quat', 
    # 'Milk_to_robot0_eef_pos', 
    # 'Milk_to_robot0_eef_quat', 
    # 'Bread_pos', 
    # 'Bread_quat', 
    # 'Bread_to_robot0_eef_pos', 
    # 'Bread_to_robot0_eef_quat', 
    # 'Cereal_pos', 
    # 'Cereal_quat', 
    # 'Cereal_to_robot0_eef_pos', 
    # 'Cereal_to_robot0_eef_quat', 
    # 'Can_pos', 
    # 'Can_quat', 
    # 'Can_to_robot0_eef_pos', 
    # 'Can_to_robot0_eef_quat',
]
for observable in useless_observable_list:
    rsenv.modify_observable(observable, 'enabled', False)
    rsenv.modify_observable(observable, 'active', False)

print('Robosuite environment maked:',type(rsenv) , rsenv, dir(rsenv))
print(len(rsenv._observables.keys()))
print(rsenv._observables.keys())

env = GymWrapper(
    rsenv
)

env.reset(seed=0)

new_env = Monitor(env)


[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /research/rs4tmr/robosuite/scripts/setup_macros.py (macros.py:55)


Robosuite environment maked: <class 'robosuite.environments.manipulation.tmr_pp.TmrPickPlaceCan'> <robosuite.environments.manipulation.tmr_pp.TmrPickPlaceCan object at 0x7ff04aaedeb0> ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_action_dim', '_check_grasp', '_check_robot_configuration', '_check_success', '_construct_objects', '_construct_visual_objects', '_create_camera_sensors', '_create_obj_sensors', '_create_segementation_sensor', '_destroy_sim', '_destroy_viewer', '_eef_xmat', '_eef_xpos', '_eef_xquat', '_get_observations', '_get_placement_initializer', '_gripper_to_target', '_initialize_sim', '_input2list', '_load_model', '_load_robots', '_obs_cache', '_observables', '_post_action', '

In [18]:
rsenv = suite.make(
    "TmrPickPlaceCan",
    robots="UR5e",  # use UR5e robot
    use_camera_obs=False,  # use pixel observations
    has_offscreen_renderer=True,  # needed if using pixel obs
    has_renderer=True,  # make sure we can render to the screen
    reward_shaping=True,  # use dense rewards
    control_freq=20,  # control should happen fast enough so that simulation looks smooth
)
rsenv.sim.render(width=640, height=480, camera_name='frontview', device_id=0)

array([[[164, 155, 147],
        [162, 153, 145],
        [163, 154, 146],
        ...,
        [179, 170, 163],
        [163, 154, 146],
        [181, 172, 164]],

       [[165, 156, 149],
        [163, 154, 146],
        [162, 153, 145],
        ...,
        [163, 155, 146],
        [177, 169, 160],
        [197, 188, 179]],

       [[167, 158, 151],
        [165, 156, 148],
        [163, 154, 146],
        ...,
        [185, 177, 168],
        [192, 183, 174],
        [189, 180, 171]],

       ...,

       [[129, 127, 123],
        [129, 127, 123],
        [129, 128, 124],
        ...,
        [ 86,  85,  82],
        [ 85,  84,  82],
        [ 86,  84,  82]],

       [[129, 128, 124],
        [129, 127, 123],
        [128, 127, 122],
        ...,
        [ 85,  84,  81],
        [ 85,  84,  81],
        [ 86,  85,  82]],

       [[128, 126, 121],
        [128, 125, 120],
        [127, 124, 120],
        ...,
        [ 84,  82,  79],
        [ 85,  83,  80],
        [ 85,  84,  81]]

In [6]:
print(new_env.render())

: 